<a href="https://colab.research.google.com/github/CClive01/CP540_Project_2/blob/main/Wind_Prestwick_2012_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import io
import pylab as plt
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
import plotly.graph_objects as go
from scipy.optimize import curve_fit

Mounted at /content/drive


In [2]:
#Creates the list of years we want to examine
years_list = list(range(2012, 2020))

#Creates a list of the dataframes for each year by using a loop. Process these dataframes and add them to the list "frame"
frame = []
for Year in years_list:
  filename = f'/content/drive/MyDrive/CP540 Data/Prestwick Wind 2010-2020/midas-open_uk-mean-wind-obs_dv-202107_ayrshire_01007_prestwick-gannet_qcv-1_{Year}.csv'
  df = pd.read_csv(filename, skiprows=[*range(79)], skipinitialspace=True)
  df = df.drop(df.columns[[1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]], axis=1)
  df = df.drop(df.tail(1).index, axis = 0)
  df['ob_end_time'] = pd.to_datetime(df.ob_end_time, format='%Y/%m/%d %H:%M:%S')
  df.dropna()
  df = df[df.ob_hour_count == 1] #Only use values with an hour count of 1
  df = df.sort_values('ob_end_time', ascending=True)
  df['timestamp'] = df['ob_end_time'].dt.strftime('%d/%m/%Y')
  frame.append(df)

#Create a single dataframe for all years by combining the dataframes in "Frame"
Comb2 = pd.concat(frame)

#Provide a list of dates which can be used for plots and finding average irradiance. We can vary the "freq=" value to add datapoints to the graph. Note: we may need to change the date format
spread = pd.date_range(f'2012-01-01',f'2019-12-31', freq='D').strftime("%d/%m/%Y").tolist()

#Create empty list for average wind speed within a timeframe
avgq = []
for i in spread:
  avgq.append((Comb2.loc[Comb2['timestamp'] == i, 'mean_wind_speed'].mean()))

#Create the plot and format as desired
custom_colors = '#662469'
plotdf = pd.DataFrame({'spread':spread, 'Daily Average Wind Speed':avgq})
fig = px.line(plotdf, x='spread', y='Daily Average Wind Speed', color_discrete_sequence=[custom_colors])
fig.update_layout(yaxis_title='Wind Speed // m/s', xaxis_title='Month', title='Average Daily Wind Speeds')
fig.show()

In [3]:
#Based on turbine from: https://generator-pro.co.uk/product/2000w-48v-wind-turbine-with-3-blades-and-tail-furling-mechanism/?gclid=CjwKCAjwkNOpBhBEEiwAb3MvvecNHpvp3MtTgmd5izN6KppDD4-Iwv0sy_oXSVmAuOPq8s06NtsEhBoCw20QAvD_BwE
#CoP from

import math

CoP = 0.3
L_blade = 3.2

plotdf['Wind Speed for Power Calc'] = plotdf['Daily Average Wind Speed']
plotdf.loc[plotdf['Wind Speed for Power Calc'] > 15, 'Wind Speed for Power Calc'] = 15
plotdf['Power'] = 0.5*CoP*1.293*math.pi*(L_blade**2)*plotdf['Wind Speed for Power Calc']**3
plotdf.loc[plotdf['Wind Speed for Power Calc'] < 2.5, 'Power'] = 0
plotdf.loc[plotdf['Power'] > 2000*CoP, 'Power'] = 2000*CoP

custom_colors = '#a43ba8'
fig2 = px.line(plotdf, x='spread', y='Power', color_discrete_sequence=[custom_colors])
fig2.update_layout(yaxis_title='Power Generation // W', xaxis_title='Month', title='Average Daily Power Production')
fig2.show()

In [4]:
Battery_size = 14 #kWh

#Starting at a fully charged battery, calculate the stored energy at the end of each hour accounting for collection, energy use, and initial stored energy
E = [Battery_size]
for i in range(1, len(plotdf['spread'])):
    # Calculate the new stored energy value based on the previous hour's value
    new_E = E[i-1] - 0.57 + plotdf['Power'].iloc[i-1]/1000
    if new_E > E[0]:
      new_E = E[0]
    E.append(new_E)

#Create plot
custom_colors = '#a43ba8'
fig3 = px.line(x=plotdf['spread'], y=E, color_discrete_sequence=[custom_colors])
fig3.update_layout(yaxis_title='Stored Energy // kWh', xaxis_title='Date', title='Stored Energy in Batteries For The 15 Year Period')
fig3.show()

In [8]:
#Create an estimate, not optimised but indicative
Battery_size = 2.5 #kWh
Panel_area = 8 #m2
Irradiance = 200 #W/m2/h

Panel_production = 0.15*Panel_area*(Irradiance/1000)

#Starting at a fully charged battery, calculate the stored energy at the end of each hour accounting for collection, energy use, and initial stored energy
E = [Battery_size]
for i in range(1, len(plotdf['spread'])):
    # Calculate the new stored energy value based on the previous hour's value
    new_E = E[i-1] - 0.57 + plotdf['Power'].iloc[i-1]/1000 + Panel_production
    if new_E > E[0]:
      new_E = E[0]
    E.append(new_E)

#Create plot
custom_colors = '#cf4e2d'
fig3 = px.line(x=plotdf['spread'], y=E, color_discrete_sequence=[custom_colors])
fig3.update_layout(yaxis_title='Stored Energy // kWh', xaxis_title='Date', title='Stored Energy in Batteries For The 15 Year Period')
fig3.show()

In [6]:
#Turbine cost data from https://www.renewableenergyhub.co.uk/main/wind-turbines/how-much-does-a-wind-turbine-cost

Installed_c = 12500 #installed cost of a 2kWh rated turbine
battery_cost = 3008 #cost of a 2.5kWh battery
Solar_panels = 250*Panel_area #using 'Panel_area' to equalise power production
c = Installed_c + battery_cost + Solar_panels
print("The total cost is £",c)

The total cost is £ 17508
